In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

import ast

In [2]:
link_df = pd.read_csv('link_detail.csv')
tthc_df = pd.read_csv('tthc_detail.csv', dtype={'maThuTuc': str})

In [3]:
missing_list = tthc_df[(tthc_df['maThuTuc']!=tthc_df['link'].str.split('=').str[-1]) | 
                        (tthc_df['maThuTuc'].str.strip()=='Không có thông tin') |
                        (~tthc_df['link'].str.contains('ma_thu_tuc'))].link.to_list()
error_list = [m for m in missing_list if 'ma_thu_tuc' not in m]
recrawl_list = [m for m in missing_list if m not in error_list]
error_list

['https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=10458&row_limit=1',
 'https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=10464&row_limit=1',
 'https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=10467&row_limit=1',
 'https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=10462&row_limit=1',
 'https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=10463&row_limit=1',
 'https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=10465&row_limit=1',
 'https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=10466&row_limit=1',
 'https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=10468&row_limit=1',
 'https://dichvucong.gov.vn/p/home/dvc-chi-tiet-cau-hoi.html?id=17271&row_limit=1']

In [4]:
# Set up the driver
driver = webdriver.Chrome()

storage = {}

for i in range(len(recrawl_list)):
    # Load the page
    driver.get(recrawl_list[i])  

    wait = WebDriverWait(driver, 3)

    main_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.col-sm-8.col-xs-12")))

    # Find all <h2> tags with class 'main-title-sub'
    headers = main_div.find_elements(By.CSS_SELECTOR, 'h2.main-title-sub')

    # Store results
    sections = []

    for header in headers:
        section_title = header.text.strip()
        content = []

        # Traverse the next sibling elements until the next header or end
        sibling = header.find_element(By.XPATH, 'following-sibling::*[1]')
        while sibling:
            # Stop if next section begins
            if 'main-title-sub' in sibling.get_attribute('class'):
                break

            content.append(sibling.get_attribute('outerHTML'))

            try:
                sibling = sibling.find_element(By.XPATH, 'following-sibling::*[1]')
            except:
                break

        sections.append({
            'section_title': section_title,
            'content': '\n'.join(content)
        })

    # Output the results
    temp = {}
    for section in sections:
        soup = BeautifulSoup(section['content'], 'html.parser')
        temp[section['section_title']] = soup.get_text().strip()

    storage[recrawl_list[i]] = temp

    print(f"Done of {len(recrawl_list)}: {i+1}", end='\r')

driver.quit()

In [5]:
df = pd.DataFrame(storage).T.reset_index().rename(columns={'index': 'link', 
'Trình tự thực hiện': 'trinhTuThucHien', 
'Cách thức thực hiện': 'cachThucThucHien',
'Thành phần hồ sơ': 'thanhPhanHoSo',
'Cơ quan thực hiện': 'coQuanThucHien',
'Yêu cầu, điều kiện thực hiện': 'yeuCauDieuKienThucHien'})
df.to_csv('tthc_recrawl.csv', index=False)
df.sample(5)

,link,trinhTuThucHien,cachThucThucHien,thanhPhanHoSo,coQuanThucHien,yeuCauDieuKienThucHien
18,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Tổ chức phát hành gửi hồ sơ đăng ký chào bán t...,"Hình thức nộp\nThời hạn giải quyết\nPhí, lệ ph...",Bao gồm\n \n\n\n\n\nTên giấy tờ\nMẫu đơ...,Ủy ban Chứng khoán Nhà nước - Bộ tài chính,
22,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Người nộp thuế (NNT) chuẩn bị hồ sơ và đến nộp...,"Hình thức nộp\nThời hạn giải quyết\nPhí, lệ ph...",Bao gồm\n \n\n\n\n\nTên giấy tờ\nMẫu đơ...,Cục thuế,Không có thông tin
36,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"1. Chủ đầu tư gửi hồ sơ, tài liệu đến Kho bạc ...","Hình thức nộp\nThời hạn giải quyết\nPhí, lệ ph...",Bao gồm\n \n\n\n\n\nTên giấy tờ\nMẫu đơ...,Kho bạc Nhà nước Việt Nam - Bộ tài chính,
72,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Bước 1: Người nộp thuế nộp hồ sơ đề nghị giảm ...,"Hình thức nộp\nThời hạn giải quyết\nPhí, lệ ph...",Bao gồm\n \n\n\n\n\nTên giấy tờ\nMẫu đơ...,Chi cục Hải quan,
3,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Người nộp thuế (NNT) chuẩn bị hồ sơ và đến nộp...,"Hình thức nộp\nThời hạn giải quyết\nPhí, lệ ph...",Bao gồm\n \n\n\n\n\nTên giấy tờ\nMẫu đơ...,"Chi cục thuế huyện, thành phố",
